In [1]:
!ls /raid/home/rajivratn/hemant_rajivratn/last/data

input.wav		 ljspeechmanifest.tsv  transcription.txt
librispeech-lm-norm.txt  train.ipynb


In [2]:
import os
import wave
import soundfile as sf
import librosa

# Directory containing the .wav files
audio_dir = "/raid/home/rajivratn/hemant_rajivratn/ljspeech/LJSpeech-1.1/wavs"
manifest_file = "/raid/home/rajivratn/hemant_rajivratn/last/data/ljspeechmanifest.tsv"

# Open the manifest file in write mode
with open(manifest_file, 'w') as f:
    # Write the directory path (you can modify as needed)
    f.write(f"{audio_dir}\n")

    # Loop through all files in the audio directory
    for filename in os.listdir(audio_dir):
        if filename.endswith('.wav'):
            # Full file path
            file_path = os.path.join(audio_dir, filename)
            
            # Read the waveform and sample rate
            waveform, sample_rate = sf.read(file_path)
            
            # Check if sample rate is 16000, if not resample
            if sample_rate != 16000:
                waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr=16000)
                sample_rate = 16000  # After resampling, set sample rate to 16000
            
            # Overwrite the original file with the resampled audio
            sf.write(file_path, waveform, sample_rate)
            
            # Get the number of samples (frames)
            num_samples = len(waveform)
            
            # Write filename and number of samples to the manifest
            f.write(f"{filename}\t{num_samples}\n")

print("Manifest file and resampled audio (overwritten) saved successfully.")

Manifest file and resampled audio (overwritten) saved successfully.
